This notebook demonstrates how to use RLRom for testing, monitoring and training models for the Cart Pole environment.

# Basic Testing and Training

RLRom works with YAML configuration files. The simplest application is to load and executing episodes of a given environment with random actions. E.g., with the file `cfg0_min.yml` which has the following content:
```YAML
env_name: CartPole-v1

cfg_test: 
  init_seeds: 0
  num_ep: 5
  num_steps: 100 
  render_mode: 'human'
```
This will execute 5 episodes of the Cart Pole environment, with 100 steps each. To do it, we can either use the CLI: 
```SH
$ rlrom_test cfg0_min.yml 
```
or create a `RLTester` object and run the `run_cfg_test()` method.

In [ ]:
from rlrom.testers import RLTester
Tt = RLTester('cfg0_min.yml')
Tres = Tt.run_cfg_test()

.....


To make things more interesting, RLRom can load a model to run with the environment. One convenient way to find a model is to get it from Huggingface. The `find_huggingface_models` connects with their API and tries to locate compatible models.

In [ ]:
from rlrom.utils import find_huggingface_models
_, model_names = Tt.find_huggingface_models(repo_contains='sb3') 
print(model_names)

['sb3/ppo-CartPole-v1', 'sb3/a2c-CartPole-v1', 'sb3/trpo-CartPole-v1', 'sb3/qrdqn-CartPole-v1', 'sb3/ars-CartPole-v1', 'sb3/dqn-CartPole-v1', 'nsanghi/dqn-cart-pole-sb3', 'nsanghi/ppo-cart-pole-sb3', 'Sumegh20/dqn-cart-pole-sb3', 'SubhasishSaha/dqn-cart-pole-sb3', 'SubhasishSaha/ppo-cart-pole-sb3', 'Sumegh20/ppo-cart-pole-sb3', 'Sumegh20/dqn-cart-pole-sb3-v2', 'Sumegh20/ppo-cart-pole-sb3-v2', 'RamaKh/dqn-cart-pole-sb3', 'Nerys249/a2c_sb3_cartpole', 'kary4FOP/a2c_sb3_cartpole', 'ThinClam/dqn-cart-pole-sb3', 'jacobvm04/cartpole-sb3-ppo', 'jacobvm04/cartpole-sb3-dqn', 'jacobvm04/cartpole-sb3-a2c']


The file `cfg0_hug.yml`specifies the first one as model to use: 

```YAML
env_name: CartPole-v1
model_path: huggingface
model_name: sb3/ppo-CartPole-v1

cfg_test: 
  init_seeds: 0
  num_ep: 5
  num_steps: 100 
  render_mode: 'human'
```


In [ ]:
Tester = RLTester('cfg0_hug.yml')
Tres = Tester.run_cfg_test()

== CURRENT SYSTEM INFO ==
- OS: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.39 # 1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
- Python: 3.13.7
- Stable-Baselines3: 2.7.0
- PyTorch: 2.8.0+cu128
- GPU Enabled: True
- Numpy: 2.3.3
- Cloudpickle: 3.1.1
- Gymnasium: 1.2.0
- OpenAI Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.0-97-generic-x86_64-with-glibc2.35 # 107-Ubuntu SMP Wed Feb 7 13:26:48 UTC 2024
- Python: 3.10.9
- Stable-Baselines3: 2.3.0a3
- PyTorch: 2.2.0+cpu
- GPU Enabled: False
- Numpy: 1.24.4
- Cloudpickle: 3.0.0
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2

.....


If there is no model available or if we need to train a new one, it is possible to do it with RLRom. The file `cfg0tr_ppo.yml` provides a basic configuration for training with ppo: 

```yaml
env_name: CartPole-v1
model_path: ./models
model_name: vanilla_ppo

cfg_test: ...

cfg_train:
  n_envs: 4
  total_timesteps: 100_000  
  algo:
    ppo: cartpole_ppo.yml
```
where `cartpole_ppo.yml` is a file containing hyper-parameters for the PPO implementation of stable-baselines3. The training can be run with 
```SH
$ rlrom_test cfg0_min.yml 
```
or using the `RLTrainer` class. 

In [22]:
from rlrom.trainers import RLTrainer
Trainer = RLTrainer('cfg0tr_ppo.yml')  

loading field [ ppo ] from YAML file [ cartpole_ppo.yml ]


In [24]:
Trainer.train()

Output()

The same configuration can be used to train and test (provided it has both a cfg_test and a cfg_train section).

# Monitoring STL Formulas

So far, the testing and training is not so different from what is done usually with SB3, e.g., in the RL-Zoo. What makes RLRom different is the ability to monitor the behavior of agents using Signal Temporal Logic, provided by the tool STLRom. 

RLRom can read STLRom `.stl` files such as `cartpole.stl` which is listed below:
```text
signal push,x,x_dot,theta,theta_dot,reward

param epsi= 0.03  
pole_up := abs(theta[t]) < epsi
keep_up := alw_[0,100] pole_up
phi_goal_keep_up := ev_[0, 400] keep_up

(...)
```

Here we formalize the goal of the problem which is that eventually, the angle must remain small (i.e., the pole must be up) for at least 100 steps. 

To monitor this goal, we add  a section `cfg_specs` In the file `cfg0_ppo_specs.yml`. The goal is to map action and observation of the environment to the signals of the STL formula. Then we specify which formula we want to evaluate and when. 

```YAML
cfg_specs:
  specs: cartpole.stl
  
  action_names:      
    push: "action"         
  
  obs_names:
    x: "obs[0]"
    x_dot: "obs[1]"
    theta: "obs[2]"
    theta_dot: "obs[3]"  
  
  eval_formulas:
    pole_up:  # we evaluate this online at all steps
      past_horizon: 0 
      eval_all_steps: true

    phi_goal_pole_up: # we evaluate this on the complete episode
      t0: 0.
      eval_all_steps: false

```
where `cartpole.stl` is an STLRom file specifying simple STL formulas. 

In [ ]:
Tester_specs = RLTester('cfg0_ppo_specs.yml')
test_results = Tester_specs.run_cfg_test()
Tester_specs.print_res_all_ep(test_results) # prints a summary of test results, including satisfaction of evaluation formulas

loading field [ specs ] from STL file [ cartpole.stl ]
loading field [ ppo ] from YAML file [ cartpole_ppo.yml ]
INFO: Loading model  /home/alex/workspace/rlrom/examples/cartpole/models/vanilla_ppo.zip
loading PPO model succeeded
..........|

mean_ep_len: 500 | mean_ep_rew: 500
phi_goal_keep_up: ratio_sat= 0.5
pole_up: sum= 5.618 | mean= 0.01124 | num_sat= 394.2


In [57]:
print(test_results['res']['phi_goal_keep_up'])

{'init_rob': array([-0.00543432, -0.02057408,  0.00197402,  0.00248138, -0.00096995,
        0.00238767, -0.00324865, -0.00076108,  0.00219337,  0.00366328]), 'init_sat': array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1])}


The results indicate the number of episodes that passed, i.e., satisfy the formula `phi_goal_keep_up`. Note that the parameter `epsi` can be changed to make the problem more or less challenging.

# Plotting

RLRom provides some convenient plotting capabilities based on the Bokeh module.

In [60]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In the previous test results, we defined and computed several formulas and episodes. The evolution of the each signal for each episode, as well as the online robust and Boolean satisfaction can be plotted on different horizontal axis. The layout is defined by a string where each line is an ax, and contains comma separated names of signals or formulas. In addition, the keyword `_ep(i)` allows to specify the current episode to plot. Below we plot traces of episode 3.

In [66]:
layout = """
 _ep(3), theta, _ep(4), theta
 _ep(3), sat(pole_up), sat(keep_up) 
 _ep(4), sat(pole_up), sat(keep_up) 
"""
fig, _=  Tester_specs.get_fig(layout)
show(fig)

_ep(3)
theta
_ep(4)
theta
_ep(3)
sat(pole_up)
sat(keep_up)
_ep(4)
sat(pole_up)
sat(keep_up)


# Training with Specifications

RLRom can also include STL formulas monitoring in the training process. We demonstrate this by training an agent satisfying a new goal specified by the following formulas:
```text
cart_on_left := x[t] < -0.5 and x[t]> -0.7
keep_left := alw_[0, 100] cart_on_left
phi_left_goal := ev_[0,400] keep_left
```
In the configuration file, we make the following changes for training.

```YAML
env_name: CartPole-v1
model_path: ./models
model_name: ppo_specs
model_use_specs: true # this means the model uses/requires specs monitoring

cfg_specs:
  specs: cartpole.stl
  action_names: ...  # same as before   
  obs_names: ...     # id.
    
  reward_formulas: 
    cart_on_left: # the robustness of this formula is *added* to reward
      past_horizon: 0 
      weight: 10  
      lower_bound: 0 # if robustness is negative, we don't add it
      
  eval_formulas:
    cart_on_left:
      past_horizon: 0
      eval_all_steps: true
    phi_left_goal:

cfg_test: ...
cfg_train:
  n_envs: 4
  eval_freq: 10000          # we eval STL formulas every 10k steps
  total_timesteps: 200_000  
  algo:
    ppo: cartpole_ppo.yml

```


In [ ]:
Tr_left= RLTrainer('cfg0tr_ppo_specs.yml')
Tr_left.train()

In [72]:
Tester_left = RLTester('cfg0tr_ppo_specs.yml')
test_results_left = Tester_left.run_cfg_test()
Tester_left.print_res_all_ep(test_results_left) # prints a summary of test results, including satisfaction of evaluation formulas

loading field [ specs ] from STL file [ cartpole.stl ]
loading field [ ppo ] from YAML file [ cartpole_ppo.yml ]
INFO: Loading model  /home/alex/workspace/rlrom/examples/cartpole/models/ppo_specs.zip
loading PPO model succeeded
..........|

mean_ep_len: 451.2 | mean_ep_rew: 812.8
cart_on_left: sum= 189.7 | mean= 0.3506 | num_sat= 405
phi_left_goal: ratio_sat= 0.9


In [73]:
lay = """
_ep(0)
 x
 sat(cart_on_left)
 reward
"""
fig, _= Tester_left.get_fig(lay)
show(fig)

_ep(0)
x
sat(cart_on_left)
reward
